# Sankey diagram of connections in trading between continents

This can be used in the future to show trading between continents and emissions consequences.

In [1]:
#Use python 3.11
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns

In [2]:
# Load data
df = pd.read_csv(r'..\data\continent_trade_matrix_CO2eq.csv')

In [3]:
print(df.year.max(), df.year.min())

2022 1986


In [4]:
#Drop antarctica
df = df[df['target']!= 'antarctica']
df = df[df['source']!= 'antarctica']

In [5]:
unique_continents = sorted(set(df['source'].unique()) | set(df['target'].unique()))

In [6]:
hex_colors = [
    "#4E79A7",  # Blue, Africa
    "#EDC948",  # Yellow, Asia
    "#E15759",  # Red, Europe
    "#76B7B2",  # Light Blue, North America
    "#B07AA1",  # Purple, Oceania
    "#59A14F",  # Green, South America
]

In [7]:
# Assuming your DataFrame is named 'df'
year = 2022
df_year = df[df['year'] == year]

# Extract unique continents for labels
unique_continents = sorted(set(df_year['source'].unique()) | set(df_year['target'].unique()))

# Create mappings to node indices (0, 1, 2, ...)
continent_to_index = {continent: i for i, continent in enumerate(unique_continents)}

# Prepare data for Sankey diagram
nodes = dict(label=unique_continents, color= hex_colors)  # Customize colors
links = dict(
    source=df_year['source'].map(continent_to_index),
    target=df_year['target'].map(continent_to_index),
    value=df_year['value'],
)

# Adjust link colors based on source node color
opacity = 0.4
link_colors = [nodes['color'][src].replace(")", f", {opacity})") for src in links['target']]

# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node= nodes,  # Customize colors,
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value'],
        color=link_colors  # Use adjusted colors
    )
)])

fig.update_layout(
    title_text=f"{year} kg CO2eq Produced by Food Trade by Continent",
    font_size=16,
    )

fig.show()


## Interactive inside output

In [ ]:
hex_colors = [
    "#4E79A7",  # Blue, Africa
    "#F28E2B",  # Orange, Asia
    "#E15759",  # Red, Europe
    "#76B7B2",  # Light Blue, North America
    "#59A14F",  # Green, Oceania
    "#EDC948",  # Yellow, South America
    "#B07AA1",  # Purple, Not assigned (extra color)
    "#FF9DA7",  # Light Red, Not assigned (extra color)
    "#9C755F",  # Brown, Not assigned (extra color)
    "#BAB0AC"   # Gray, Not assigned (extra color)
]

In [84]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'df' is your DataFrame and contains columns 'year', 'source', 'target', and 'value'
# List of unique years in the dataset
years = df['year'].unique()

# Create an empty list to hold the frames for the animation
frames = []

# Prepare unique continents and colors (customize as needed)
unique_continents = sorted(set(df['source'].unique()) | set(df['target'].unique()))
continent_to_index = {continent: i for i, continent in enumerate(unique_continents)}

# Generate color palette
#palette = sns.color_palette("colorblind", len(unique_continents))
# Convert Seaborn colors to hex colors
#hex_colors = palette.as_hex()

hex_colors = [
    "#4E79A7",  # Blue, Africa
    "#EDC948",  # Yellow, Asia
    "#E15759",  # Red, Europe
    "#76B7B2",  # Light Blue, North America
    "#B07AA1",  # Purple, Oceania
    "#59A14F",  # Green, South America
]

# Function to create link colors with opacity
def adjust_link_colors(source_indices, colors, opacity=0.2):
    return [colors[src].replace(")", f", {opacity})") for src in source_indices]

# Create a frame for each year
for year in years:
    df_year = df[df['year'] == year]
    nodes = dict(label=unique_continents, color=hex_colors)
    links = dict(
        source=df_year['source'].map(continent_to_index),
        target=df_year['target'].map(continent_to_index),
        value=df_year['value']
    )
    link_colors = adjust_link_colors(links['target'], hex_colors)
    
    frame = go.Frame(
        data=[go.Sankey(
            node=nodes,
            link=dict(
                source=links['source'],
                target=links['target'],
                value=links['value'],
                color=link_colors
            )
        )],
        name=str(year),
        layout=go.Layout(title_text=f"{year} kg CO2eq Produced by Food Trade by Continent")
    )
    frames.append(frame)

# Create initial plot
initial_year = years[0]
df_initial = df[df['year'] == initial_year]
nodes_initial = dict(label=unique_continents, color=hex_colors)
links_initial = dict(
    source=df_initial['source'].map(continent_to_index),
    target=df_initial['target'].map(continent_to_index),
    value=df_initial['value']
)
link_colors_initial = adjust_link_colors(links_initial['target'], hex_colors)

fig = go.Figure(
    data=[go.Sankey(
        node=nodes_initial,
        link=dict(
            source=links_initial['source'],
            target=links_initial['target'],
            value=links_initial['value'],
            color=link_colors_initial
        )
    )],
    frames=frames
)

# Add animation settings
fig.update_layout(
    updatemenus=[dict(type="buttons", showactive=False, buttons=[dict(label="Play",
                      method="animate", args=[None, dict(frame=dict(duration=600, redraw=True), 
                                                        fromcurrent=True)])])],
    sliders=[dict(
        steps=[dict(method='animate', args=[[str(year)], dict(mode='immediate', frame=dict(duration=1000, redraw=True), transition=dict(duration=300))],
                    label=str(year)) for year in years],
        transition=dict(duration=300),
        x=0.1,
        xanchor="left",
        y=0,
        yanchor="top"
    )],
    title_text=f"{initial_year} kg CO2eq Produced by Food Trade by Continent",
    font_size=16
)

fig.show()

## Save to chart studio

In [85]:
import chart_studio 

username = "ezemriv"
password = "dmanBuRgSIAPjUFi4m1X"

chart_studio.tools.set_credentials_file(username=username, api_key=password)

import chart_studio.plotly as py
import chart_studio.tools as tls

In [86]:
py.plot(fig, filename = 'Interactive_Sankey_CO2eq_trade_continents', auto_open=False)

'https://plotly.com/~ezemriv/7/'

# One plot every 10 years from 2022 back

In [100]:
# Assuming your DataFrame is named 'df'
# Create a list of years from 2022 backwards in steps of 10 years
years = list(range(2022, df['year'].min() - 1, -10))

# Filter out years that are not in the DataFrame
years = [year for year in years if year in df['year'].unique()]

# Loop through each year and create Sankey diagrams
for year in years:
    df_year = df[df['year'] == year]

    # Extract unique continents for labels
    unique_continents = sorted(set(df_year['source'].unique()) | set(df_year['target'].unique()))

    # Create mappings to node indices (0, 1, 2, ...)
    continent_to_index = {continent: i for i, continent in enumerate(unique_continents)}

    # Prepare data for Sankey diagram
    nodes = dict(label=unique_continents, color=hex_colors)  # Customize colors
    links = dict(
        source=df_year['source'].map(continent_to_index),
        target=df_year['target'].map(continent_to_index),
        value=df_year['value'],
    )

    # Adjust link colors based on source node color
    opacity = 0.4
    link_colors = [nodes['color'][src].replace(")", f", {opacity})") for src in links['target']]

    # Create the Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=nodes,  # Customize colors,
        link=dict(
            source=links['source'],
            target=links['target'],
            value=links['value'],
            color=link_colors  # Use adjusted colors
        )
    )])

    fig.update_layout(
        title_text=f"{year} kg CO2eq Produced by Food Trade by Continent",
        font_size=16,
    )

    # Save each figure to an HTML file
    fig.write_html(f"sankey_{year}.html")